In [ ]:
# Install requirements
!pip install transformers==4.44.0 peft==0.10.0 torch accelerate bitsandbytes

   # Clone repo and get sample data
!git clone https://github.com/zzachw/llemr.git
!cp llemr/sample_data/qa_event_subset.jsonl .
!cp llemr/sample_data/qa_note_subset.jsonl .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import re

def init_model(model_name):
    print("Loading model and tokenizer...")

    # Load model without quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        trust_remote_code=True
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        padding_side="left",
        trust_remote_code=True
    )

    # Set pad token if not set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer

def format_prompt(question, patient_data):
    # Organize patient data by event type
    events_by_type = {}
    for item in patient_data:
        event_type = item['event_type']
        if event_type not in events_by_type:
            events_by_type[event_type] = []
        events_by_type[event_type].append(item)

    # Extract time information from the question if present
    time_pattern = r'at (?:the )?(\d+\.?\d*)[- ]?(?:hour|hours|hr|hrs)?'
    time_match = re.search(time_pattern, question.lower())
    target_time = float(time_match.group(1)) if time_match else None

    # Function to extract key terms from a question
    def extract_key_terms(text):
        # Convert to lowercase and split into words
        words = text.lower().split()
        # Remove common words and punctuation
        stop_words = {'what', 'which', 'where', 'when', 'how', 'was', 'were', 'did', 'does', 'do', 'the', 'a', 'an', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}
        key_terms = [word.strip('.,?!') for word in words if word.strip('.,?!') not in stop_words]
        return set(key_terms)

    # Extract key terms from the user's question
    question_terms = extract_key_terms(question)

    # Find matching questions based on key terms and time
    matching_items = []
    for item in patient_data:
        item_terms = extract_key_terms(item['q'])
        # Calculate term overlap
        overlap = len(question_terms.intersection(item_terms))
        if overlap >= 2:  # If at least 2 key terms match
            matching_items.append((overlap, item['q'], item['a']))  # Store score, question and answer as tuple

    # Sort by overlap score (first element of tuple)
    matching_items.sort(key=lambda x: x[0], reverse=True)

    # Build context with organized information
    context = "Patient Information:\n"

    # If we have matching items, prioritize them
    if matching_items:
        context += f"\nRELEVANT INFORMATION:\n"
        # Include up to 2 most relevant matches
        for _, q, a in matching_items[:2]:
            context += f"Q: {q}\nA: {a}\n"
        context += "\n"

    # Priority order for event types
    priority_events = [
        'patient_demographics',
        'admission_info',
        'diagnoses_icd',
        'procedureevents',
        'labevents',
        'prescriptions',
        'microbiologyevents',
        'transfers'
    ]

    # Add information in priority order
    for event_type in priority_events:
        if event_type in events_by_type:
            context += f"\n{event_type.upper()}:\n"
            items = events_by_type[event_type]

            # If there's a time in the question and this is a time-sensitive event type
            if target_time is not None and event_type in ['labevents', 'procedureevents', 'prescriptions', 'microbiologyevents']:
                # Include all items that match the specific time
                time_relevant_items = []
                for item in items:
                    if str(target_time) in item['q']:
                        time_relevant_items.append(item)

                # If we found time-specific items, only show those
                if time_relevant_items:
                    items = time_relevant_items

            # Show all relevant items for the event type
            shown_items = set()  # Track items we've already shown
            for item in items:
                # Check if this item's question is not in the matching items
                if item['q'] not in [m[1] for m in matching_items]:
                    item_text = f"- {item['q']} {item['a']}\n"
                    if item_text not in shown_items:
                        context += item_text
                        shown_items.add(item_text)

    # Create the full prompt with more explicit instructions
    return f"""<|system|>You are a medical AI assistant. Answer questions based ONLY on the following patient information. If the specific information needed is not in the context, say "I don't have enough information to answer this question." Be concise and direct. If the question asks about a specific time, only use information from that exact time. DO NOT repeat the context in your answer - instead, provide a clear and direct response to the question.</s>
<|user|>
{context}

Question: {question}

Provide a direct answer without repeating the context:</s>
<|assistant|>"""

def main():
    # Free up memory
    import gc
    gc.collect()
    torch.cuda.empty_cache()

    # Initialize model and tokenizer - using a smaller model
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

    model, tokenizer = init_model(model_name)
    model.eval()

    # Get the device where the model is located
    device = next(model.parameters()).device
    print(f"Model is on device: {device}")

    # Set up generation config with stricter parameters
    generation_config = GenerationConfig(
        do_sample=True,
        temperature=0.1,  # Even lower temperature for more focused responses
        top_p=0.5,       # More restrictive top_p
        max_new_tokens=50,  # Shorter responses
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2  # Penalize repetition
    )

    # Load sample data
    print("Loading sample data...")
    try:
        with open("sample_data/qa_event_subset.jsonl", "r") as f:
            event_data = [json.loads(line) for line in f]
    except FileNotFoundError:
        try:
            with open("qa_event_subset.jsonl", "r") as f:
                event_data = [json.loads(line) for line in f]
        except FileNotFoundError:
            print("Error: Could not find the data file in either location")
            return

    # Get unique patient IDs
    patient_ids = sorted(list(set(item['hadm_id'] for item in event_data)))
    print(f"\nAvailable patient IDs: {patient_ids[:5]} ...")

    # Show example questions for first patient
    print(f"\nExample questions for patient {patient_ids[0]}:")
    example_questions = [item for item in event_data if item['hadm_id'] == patient_ids[0]][:3]
    for q in example_questions:
        print(f"- {q['q']}")

    while True:
        patient_id = input("\nEnter patient ID (or 'quit' to exit): ")
        if patient_id.lower() == 'quit':
            break

        try:
            patient_id = int(patient_id)
            patient_questions = [item for item in event_data if item['hadm_id'] == patient_id]
            if not patient_questions:
                print(f"No questions found for patient {patient_id}")
                continue

            print(f"\nFound {len(patient_questions)} questions for patient {patient_id}")
            question = input("Enter your question (or 'list' to see available questions): ")

            if question.lower() == 'list':
                # Organize questions by event type
                questions_by_type = {}
                for i, q in enumerate(patient_questions):
                    event_type = q['event_type']
                    if event_type not in questions_by_type:
                        questions_by_type[event_type] = []
                    questions_by_type[event_type].append((i+1, q))

                # Display questions organized by type
                for event_type, questions in questions_by_type.items():
                    print(f"\n{event_type.upper()}:")
                    for idx, q in questions:
                        print(f"{idx}. Q: {q['q']}\n   A: {q['a']}\n")
                continue

            # Format input with chat template and patient context
            input_text = format_prompt(question, patient_questions)

            # Generate response
            inputs = tokenizer(input_text, return_tensors="pt")
            # Move inputs to the same device as the model
            inputs = {k: v.to(device) for k, v in inputs.items()}

            try:
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        generation_config=generation_config
                    )
                # Move outputs back to CPU for decoding
                outputs = outputs.cpu()
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)

                # Clean up the response - get only the assistant's part
                if "<|assistant|>" in response:
                    response = response.split("<|assistant|>")[-1].strip()

                print(f"\nModel Answer: {response}")

                # Find if this exact question exists in the dataset
                matching_qa = next((item for item in patient_questions if item['q'].lower() == question.lower()), None)
                if matching_qa:
                    print(f"True Answer from dataset: {matching_qa['a']}")

            except Exception as e:
                print(f"Error during generation: {str(e)}")
                response = "Error: Could not generate response"

        except ValueError:
            print("Please enter a valid patient ID")

        print("\n" + "-" * 80)

if __name__ == "__main__":
    main()

Loading model and tokenizer...
Model is on device: cuda:0
Loading sample data...

Available patient IDs: [20044587, 20199380, 20214994, 20291550, 20297618] ...

Example questions for patient 20044587:
- Which interventions were carried out within the initial 48 hours?
- Can you identify the three primary diagnoses billed for the patient?
- Can you confirm the patient's marital status?

Found 6 questions for patient 20199380

LABEVENTS:
1. Q: Which Hematology tests were conducted on the blood sample at the 50.92-hour mark?
   A: The blood specimen was analyzed for Hematocrit, Hemoglobin, MCH, MCHC, MCV, Platelet Count, RDW, Red Blood Cells, White Blood Cells, RDW-SD, INR(PT), PT, and PTT.


PRESCRIPTIONS:
2. Q: What dosage was in the most recent Fentanyl Citrate prescription given within the initial 48 hours?
   A: The prescription contained a 100mcg/2mL Amp of Fentanyl Citrate.


PROCEDUREEVENTS:
3. Q: Which interventions were carried out on the second day?
   A: A 20-gauge procedure w